In [3]:
using PowerSystems
using Dates
using TimeSeries
using DataFrames
using CSV
using DataStructures
using PowerSystemCaseBuilder
using PowerSimulations
using HiGHS
using HydroPowerSimulations

In [31]:
# from https://github.com/NREL-Sienna/PowerSystems.jl/blob/main/docs/src/tutorials/creating_system.md
sys = System(100.0)

bus1 = ACBus(;
    number = 1,
    name = "bus1",
    bustype = ACBusTypes.REF,
    angle = 0.0,
    magnitude = 1.0,
    voltage_limits = (min = 0.9, max = 1.05),
    base_voltage = 230.0,
);

bus2 = ACBus(;
    number = 2,
    name = "bus2",
    bustype = ACBusTypes.PV,
    angle = 0.0,
    magnitude = 1.0,
    voltage_limits = (min = 0.9, max = 1.05),
    base_voltage = 230.0,
);

bus3 = ACBus(;
    number = 3,
    name = "bus3",
    bustype = ACBusTypes.PV,
    angle = 0.0,
    magnitude = 1.0,
    voltage_limits = (min = 0.9, max = 1.05),
    base_voltage = 230.0,
);

bus4 = ACBus(;
    number = 4,
    name = "bus4",
    bustype = ACBusTypes.PV,
    angle = 0.0,
    magnitude = 1.0,
    voltage_limits = (min = 0.9, max = 1.05),
    base_voltage = 230.0,
);

line21 = Line(;
    name = "line21",
    available = true,
    active_power_flow = 0.0,
    reactive_power_flow = 0.0,
    arc = Arc(; from = bus2, to = bus1),
    r = 0.00281, # Per-unit
    x = 0.0281, # Per-unit
    b = (from = 0.00356, to = 0.00356), # Per-unit
    rating = 3.5, # Line rating of 350 MVA / System base of 100 MVA
    angle_limits = (min = -0.7, max = 0.7),
);

line31 = Line(;
    name = "line31",
    available = true,
    active_power_flow = 0.0,
    reactive_power_flow = 0.0,
    arc = Arc(; from = bus3, to = bus1),
    r = 0.00281, # Resistance, Per-unit
    x = 0.0281, # Reactance, Per-unit
    b = (from = 0.00356, to = 0.00356), # Shunt susceptance, Per-unit
    rating = 2.0, # Line rating of 200 MVA / System base of 100 MVA
    angle_limits = (min = -0.7, max = 0.7),
);

line41 = Line(;
    name = "line41",
    available = true,
    active_power_flow = 0.0,
    reactive_power_flow = 0.0,
    arc = Arc(; from = bus4, to = bus1),
    r = 0.00281, # Resistance, Per-unit
    x = 0.0281, # Reactance, Per-unit
    b = (from = 0.00356, to = 0.00356), # Shunt susceptance, Per-unit
    rating = 2.0, # Line rating of 200 MVA / System base of 100 MVA
    angle_limits = (min = -0.7, max = 0.7),
);

load =  PowerLoad(;
    name = "bus1",
    available = true,
    bus = bus1,
    active_power = 0.0, # Per-unitized by device base_power
    reactive_power = 0.0, # Per-unitized by device base_power
    base_power = 110.0, # MVA
    max_active_power = 1.0, # 10 MW per-unitized by device base_power
    max_reactive_power = 0.0,
);

# https://nrel-sienna.github.io/PowerSystems.jl/stable/model_library/generated_RenewableDispatch/#RenewableDispatch
solar = RenewableDispatch(;
    name = "SolarBusA",
    available = true,
    bus = bus2,
    active_power = 0.0, # Per-unitized by device base_power
    reactive_power = 0.0, # Per-unitized by device base_power
    rating = 1.0, # 5 MW per-unitized by device base_power
    prime_mover_type = PrimeMovers.PVe,
    reactive_power_limits = (min = 0.0, max = 0.0), # per-unitized by device base_power
    power_factor = 1.0,
    operation_cost = RenewableGenerationCost(CostCurve(LinearCurve(0.0075))),
    # https://nrel-sienna.github.io/PowerSystems.jl/stable/model_library/renewable_generation_cost/#RenewableGenerationCost
    # https://nrel-sienna.github.io/PowerSystems.jl/stable/api/public/#InfrastructureSystems.CostCurve
    # https://nrel-sienna.github.io/PowerSystems.jl/stable/api/public/#InfrastructureSystems.LinearCurve
    base_power = 330, # MVA
);

wind = RenewableDispatch(;
    name = "WindBusA",
    available = true,
    bus = bus3,
    active_power = 0.0,
    reactive_power = 0.0,
    rating = 1.0,
    prime_mover_type = PrimeMovers.WT,
    reactive_power_limits = (min = 0.0, max = 0.0),
    power_factor = 1.0,
    operation_cost = RenewableGenerationCost(CostCurve(LinearCurve(0.0075))),
    base_power = 60.0 # MVA
    )

# https://github.com/NREL-Sienna/PowerSystems.jl/blob/ec2c55991de76d25deb0c9ac23d85e44979d9694/src/models/generated/HydroEnergyReservoir.jl
hydro = HydroPumpedStorage(;
    name = "hydro",
    available = true,
    bus = bus4,
    active_power = 0.0,
    reactive_power = 0.0,
    active_power_limits = (min=0.0, max=42), # MW from 2 28,000 HP pumps https://www.sdcwa.org/wp-content/uploads/2020/11/lake_hodges_fs.pdf?q=/sites/default/files/files/publications/lakehodges-fs.pdf&
    rating = 1.0,
    prime_mover_type = PrimeMovers.PS,
    reactive_power_limits = nothing,
    rating_pump = 42, # MVA
    active_power_limits_pump = (min=0.0, max=42),
    reactive_power_limits_pump = nothing,
    ramp_limits_pump = nothing,
    time_limits_pump = nothing,
    inflow = 0,
    outflow = 0,
    initial_storage = (up=37314003, down=0),
    ramp_limits=nothing,
    time_limits=nothing,
    operation_cost = HydroGenerationCost(nothing),
    base_power = 60.0, # MVA
    storage_capacity = (up=37314003, down=37314003) # m^3, from https://www.sandiego.gov/reservoirs-lakes/hodges-reservoir
    )

add_components!(sys, [bus1, bus2, bus3, bus4, line21, line31, line41, load, solar, wind, hydro])

In [32]:
sys

Property,Value
Name,
Description,
System Units Base,SYSTEM_BASE
Base Power,100.0
Base Frequency,60.0
Num Components,14
Type,Count
ACBus,4
Arc,3
HydroPumpedStorage,1


In [26]:
get_component(PowerSystems.ElectricLoad, sys, "bus1")

PowerLoad: bus1:
   name: bus1
   available: true
   bus: ACBus: bus1
   active_power: 0.0
   reactive_power: 0.0
   base_power: 110.0
   max_active_power: 1.1
   max_reactive_power: 0.0
   services: 0-element Vector{Service}
   dynamic_injector: nothing
   ext: Dict{String, Any}()
   InfrastructureSystems.SystemUnitsSettings:
      base_value: 100.0
      unit_system: UnitSystem.SYSTEM_BASE = 0
   has_supplemental_attributes: false
   has_time_series: false

In [27]:
get_component(PowerSystems.RenewableDispatch, sys, "SolarBusA")

RenewableDispatch: SolarBusA:
   name: SolarBusA
   available: true
   bus: ACBus: bus2
   active_power: 0.0
   reactive_power: 0.0
   rating: 3.3
   prime_mover_type: PrimeMovers.PVe = 21
   reactive_power_limits: (min = 0.0, max = 0.0)
   power_factor: 1.0
   operation_cost: RenewableGenerationCost composed of variable: CostCurve{LinearCurve}, curtailment_cost: CostCurve{LinearCurve}
   base_power: 330.0
   services: 0-element Vector{Service}
   dynamic_injector: nothing
   ext: Dict{String, Any}()
   InfrastructureSystems.SystemUnitsSettings:
      base_value: 100.0
      unit_system: UnitSystem.SYSTEM_BASE = 0
   has_supplemental_attributes: false
   has_time_series: false

In [28]:
get_component(PowerSystems.Generator, sys, "WindBusA")

RenewableDispatch: WindBusA:
   name: WindBusA
   available: true
   bus: ACBus: bus3
   active_power: 0.0
   reactive_power: 0.0
   rating: 0.6
   prime_mover_type: PrimeMovers.WT = 22
   reactive_power_limits: (min = 0.0, max = 0.0)
   power_factor: 1.0
   operation_cost: RenewableGenerationCost composed of variable: CostCurve{LinearCurve}, curtailment_cost: CostCurve{LinearCurve}
   base_power: 60.0
   services: 0-element Vector{Service}
   dynamic_injector: nothing
   ext: Dict{String, Any}()
   InfrastructureSystems.SystemUnitsSettings:
      base_value: 100.0
      unit_system: UnitSystem.SYSTEM_BASE = 0
   has_supplemental_attributes: false
   has_time_series: false

In [33]:
get_component(PowerSystems.HydroPumpedStorage, sys, "hydro")

HydroPumpedStorage: hydro:
   name: hydro
   available: true
   bus: ACBus: bus4
   active_power: 0.0
   reactive_power: 0.0
   rating: 0.6
   base_power: 60.0
   prime_mover_type: PrimeMovers.PS = 18
   active_power_limits: (min = 0.0, max = 25.2)
   reactive_power_limits: nothing
   ramp_limits: nothing
   time_limits: nothing
   rating_pump: 25.2
   active_power_limits_pump: (min = 0.0, max = 25.2)
   reactive_power_limits_pump: nothing
   ramp_limits_pump: nothing
   time_limits_pump: nothing
   storage_capacity: (up = 2.23884018e7, down = 2.23884018e7)
   inflow: 0.0
   outflow: 0.0
   initial_storage: (up = 2.23884018e7, down = 0.0)
   storage_target: (up = 1.0, down = 1.0)
   operation_cost: HydroGenerationCost composed of variable: CostCurve{LinearCurve}
   pump_efficiency: 1.0
   conversion_factor: 1.0
   status: PowerSystems.PumpHydroStatusModule.PumpHydroStatus.OFF = 0
   time_at_status: 10000.0
   services: 0-element Vector{Service}
   dynamic_injector: nothing
   ext: Dict

https://nrel-sienna.github.io/PowerSystems.jl/stable/how_to/parse_ts_from_csvs/

In [ ]:
fname = joinpath(homedir(), "ecen5407", "ecen5407_project2", "data", "Sienna_Inputs", "sienna_pointers.json");
add_time_series!(sys, fname)

In [ ]:
transform_single_time_series!(
    sys,
    Dates.Hour(24), # horizon
    Dates.Hour(1), # interval
);

In [ ]:
sys

In [ ]:
solver = optimizer_with_attributes(HiGHS.Optimizer, "mip_rel_gap" => 0.5);
template_uc = template_unit_commitment();
problem = DecisionModel(template_uc, sys; optimizer = solver, name = "UC", horizon = Hour(1))
build!(problem, output_dir = mktempdir())

In [ ]:
solve!(problem)

In [ ]:
res = OptimizationProblemResults(problem)

In [ ]:
renewable_param = read_parameter(res, "ActivePowerTimeSeriesParameter__RenewableDispatch")